# Install packages + check hardware

In [ ]:
! pip install tensorflow==2.9.0

restart runtime

In [ ]:
!pip install tensorflow_model_optimization

In [ ]:
from google.colab import drive
import tensorflow as tf
import tensorflow_model_optimization as tfmot

drive.mount('/content/drive')
print("tensorflow version: ", tf.__version__) # 2.9.0
!nvidia-smi -L

# Dataset.py

In [ ]:
! unrar x /content/drive/MyDrive/Dataset/NWPU-RESISC45.rar /content/
import os 
print(len(os.listdir('/content/NWPU-RESISC45/')))
print(len(os.listdir('/content/NWPU-RESISC45/airplane')))

In [ ]:
import os
import numpy as np
from PIL import Image
import random

input_dir = '/content/NWPU-RESISC45'
store_dir = '/content/dataset2'

# check/create directory
if not os.path.exists(store_dir):
  print('==== Create Data Dir =====')
  os.makedirs(store_dir)

store_dir_train = os.path.join(store_dir, 'train')
if not os.path.exists(store_dir_train):
  print('==== Create Train Dir =====')
  os.makedirs(store_dir_train)

store_dir_test = os.path.join(store_dir, 'test')
if not os.path.exists(store_dir_test):
  print('==== Create Test Dir =====')
  os.makedirs(store_dir_test)

class_list = os.listdir(input_dir)  # Name of all classes 
class_list.sort() # sort alphetically

for class_name in class_list:
  class_train_dir = os.path.abspath(os.path.join(store_dir_train, class_name))
  os.makedirs(class_train_dir)
  class_test_dir = os.path.abspath(os.path.join(store_dir_test, class_name))
  os.makedirs(class_test_dir)

  class_input_dir = os.path.abspath(os.path.join(input_dir, class_name))
  file_name_list = os.listdir(class_input_dir)
  file_name_list.sort()
  # file_name_random = np.random.permutation(range(len(file_name_list)))

  for i in range(len(file_name_list)):
    # file_name = file_name_list[file_name_random[i]]
    file_name = file_name_list[i]
    file_open = os.path.join(class_input_dir, file_name)
    org_img = Image.open(file_open)   # take out image
    org_img = org_img.resize((256,256)) # resize
    if i >= int(len(file_name_list)*0.8): # train set
      train_path = os.path.join(class_train_dir, file_name)
      org_img.save(train_path)
    else:                                   # test  set
      test_path = os.path.join(class_test_dir, file_name)
      org_img.save(test_path)

print(len(os.listdir('/content/dataset2/train/')))
print(len(os.listdir('/content/dataset2/test/')))
print(len(os.listdir('/content/dataset2/train/airplane')))
print(len(os.listdir('/content/dataset2/test/airplane')))
l = os.listdir('/content/dataset2/train/airplane')
l.sort()
print(l[0],l[-1])

train_dir = '/content/dataset2/train'
test_dir  = '/content/dataset2/test'
train_count = 0
test_count = 0
class_list = os.listdir('/content/dataset2/train/')
class_list.sort()
for c in class_list:
  train_path = os.path.join(train_dir,c)
  test_path  = os.path.join(test_dir,c)
  train_count += len(os.listdir(train_path))
  test_count += len(os.listdir(test_path))
  if len(os.listdir(train_path)) != 140 or len(os.listdir(test_path)) != 560:
    print('something wrong on class: ', c)

print(train_count, test_count, train_count + test_count)

del train_count,test_count,train_dir,test_dir,l,org_img,file_name,file_open,file_name_list,class_list,class_input_dir,train_path,test_path,class_train_dir,class_test_dir

# Model.py

## model_1: ContrastiveLoss using probability distancce + cosine similarity

In [ ]:
pre_trained_model =  tf.keras.models.load_model('/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_contrastive_2/contrastive_model.h5')

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv2D, Activation, Dropout, Flatten, Input, Dense, MultiHeadAttention
from tensorflow.keras.layers import Add, Average, Concatenate,Reshape, multiply, Permute, Lambda
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.activations import sigmoid

class ContrastiveLoss(tf.keras.losses.Loss):
  def __init__(self, temperature=0.27, rate=0.5, name='ContrastiveLoss', **kwargs):
    super(ContrastiveLoss, self).__init__(name=name, **kwargs)
    self.temperature   = temperature
    self.rate          = rate
    self.cosine_sim    = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.NONE)
    self.cross_entropy = tf.keras.losses.CategoricalCrossentropy(axis=-1, reduction=tf.keras.losses.Reduction.NONE)
    
  # @tf.function
  def call(self, z1, z2):
    batch_size, n_dim = z1.shape

    # Compute Distance loss
    z1_soft  = tf.keras.activations.softmax(z1, axis=-1)       # (Bx1280) 
    z2_soft  = tf.keras.activations.softmax(z2, axis=-1)       # (Bx1280) 
    distance = self.cross_entropy(z1_soft, z2_soft)            # (B) 
    mean_distance = tf.sqrt(tf.reduce_mean(distance))          # () -> scalar
    tf.debugging.check_numerics(mean_distance.numpy(), 'Distance contains NaN values.')

    # Compute Consine Similarity loss
    z = tf.concat((z1, z2), 0)

    sim_ij      = - self.cosine_sim(z[:batch_size], z[batch_size:])     # (B)  -> batch_size pair
    sim_ji      = - self.cosine_sim(z[batch_size:], z[:batch_size])     # (B)  -> batch_size pair
    sim_pos     = tf.concat((sim_ij,sim_ji), axis=0)                    # (2B) -> 2*batch_size positive pair
    numerator   = tf.math.exp(sim_pos / self.temperature)               # (2B) -> 2*batch_size positive pair
  
    sim_neg     = - self.cosine_sim(tf.expand_dims(z, 1), z)            # sim (Bx1xE, BxE) -> (2Bx2B)
    mask        = 1 - tf.eye(2*batch_size, dtype=tf.float32)            # (2Bx2B)
    sim_neg     = mask * tf.math.exp(sim_neg / self.temperature)        # (2Bx2B)
    denominator = tf.math.reduce_sum(sim_neg, axis=-1)                  # (2B) 
  
    mean_cosine_similarity = tf.reduce_mean(- tf.math.log(numerator / denominator))       # () -> scalar
    tf.debugging.check_numerics(mean_cosine_similarity.numpy(), 'Cosine contains NaN values.')

    # Compute total loss with associated rate
    total_loss = (1-self.rate)*mean_distance + self.rate*mean_cosine_similarity 
    tf.debugging.check_numerics(total_loss.numpy(), 'Total contains NaN values.')
    return total_loss

def trippleAttention(x): # 8x8xc
  ## TA - Tripple Attention
  c = x.shape[-1]
    # channel
  tl1 = tf.math.reduce_mean(x, axis=-1) + tf.math.reduce_max(x, axis=-1)# 8x8        
  tl1 = MultiHeadAttention(num_heads=16, key_dim=8)(tl1, tl1)           # 8x8 
  tl1 = sigmoid(tl1)            # 8x8
  tl1 = Reshape((8,8,1))(tl1)   # 8x8x1
  tl1 = x * tl1                 # 8x8xc * 8x8x1 -> 8x8xc
    # width
  tl2 = tf.math.reduce_mean(x, axis=-2) + tf.math.reduce_max(x, axis=-2)# 8xc
  tl2 = MultiHeadAttention(num_heads=16, key_dim=8)(tl2, tl2)          # 8xc
  tl2 = sigmoid(tl2)            # 8xcx1
  tl2 = Reshape((8,1,c))(tl2)   # 8x1xc
  tl2 = x * tl2                 # 8x8xc * 8x1xc -> 8x8xc
    # height
  tl3 = tf.math.reduce_mean(x, axis=-3) + tf.math.reduce_max(x, axis=-3)# 8xc
  tl3 = MultiHeadAttention(num_heads=16, key_dim=8)(tl3, tl3)          # 8xc
  tl3 = sigmoid(tl3)            # 8xcx1
  tl3 = Reshape((1,8,c))(tl3)   # 8x1xc
  tl3 = x * tl3                 # 8x8xc * 1x8xc -> 8x8xc
    # average 
  t = Average()([tl1, tl2, tl3]) # 8x8xc
  t = GlobalAveragePooling2D(keepdims=False)(t) #channel
  return t # c

def get_model_architecture():
  base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(256,256,3))

  block7_x = base_model.output                           
  block6_x = base_model.get_layer('block6d_add').output  
  block5_x = base_model.get_layer('block5c_add').output  
  
  block6_x = Conv2D(filters=1280, kernel_size=1, strides=1)(block6_x) 
  block5_x = Conv2D(filters=1280, kernel_size=2, strides=2)(block5_x) 
  
  block7_x = trippleAttention(block7_x)
  block6_x = trippleAttention(block6_x)
  block5_x = trippleAttention(block5_x)
  
  x = block5_x + block6_x + block7_x
  x = Dense(512, 
                    activation='relu',
                    kernel_initializer=initializers.TruncatedNormal(mean=0.0,stddev=0.1),
                    kernel_regularizer=regularizers.l2(1e-5),
                    bias_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.1),
                    bias_regularizer=regularizers.l2(1e-5)
                    )(x)
  x = Dropout(0.2)(x)
  predictions = Dense(45, 
                        activation='softmax',
                        kernel_initializer=initializers.TruncatedNormal(mean=0.0,stddev=0.1),
                        kernel_regularizer=regularizers.l2(1e-5),
                        bias_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.1),
                        bias_regularizer=regularizers.l2(1e-5)
                        )(x)
  return Model(base_model.input, predictions)

# create full model
def get_cls_model():
  model = get_model_architecture()
  model.set_weights(pre_trained_model.get_weights()) 
  model.trainable = True 
  opt = tf.keras.optimizers.Adam(learning_rate=1e-6) 
  model.compile(loss=[tf.keras.losses.CategoricalCrossentropy(),ContrastiveLoss()], optimizer=opt, metrics=[])   
  return model

if os.path.exists('/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_567_con_2loss/contrastive_model.h5'):
  print('loading model !')
  model = tf.keras.models.load_model( filepath='/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_567_con_2loss/contrastive_model.h5', 
                                      custom_objects={'ContrastiveLoss': ContrastiveLoss})
  print(model.loss)
  print(model.optimizer.learning_rate)
  # model.summary()
else:
  print('creating model !')
  model = get_cls_model()
  model.summary()


## model_2: ContrastiveLoss using Euclid distancce + cosine similarity

In [ ]:
import tensorflow as tf

class ContrastiveLoss(tf.keras.losses.Loss):
  def __init__(self, temperature=0.27, rate=0.72, name='ContrastiveLoss', **kwargs):
    super(ContrastiveLoss, self).__init__(name=name, **kwargs)
    self.temperature   = temperature
    self.rate          = rate
    self.cosine_sim    = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.NONE)
    self.cross_entropy = tf.keras.losses.CategoricalCrossentropy(axis=-1, reduction=tf.keras.losses.Reduction.NONE)
    
  # @tf.function
  def call(self, z1, z2):
    batch_size, n_dim = z1.shape

    # Compute Distance loss
    z1_soft = tf.keras.activations.softmax(z1, axis=-1)       # (Bx1280) 
    z2_soft = tf.keras.activations.softmax(z2, axis=-1)       # (Bx1280) 
    distance = self.cross_entropy(z1_soft, z2_soft)           # (B) 
    mean_distance = tf.sqrt(tf.reduce_mean(distance))         # () -> scalar
    # print('dis: ', mean_distance)
    tf.debugging.check_numerics(mean_distance.numpy(), 'Distance contains NaN values.')

    # Compute Consine Similarity loss
    z = tf.concat((z1, z2), 0)

    sim_ij      = - self.cosine_sim(z[:batch_size], z[batch_size:])     # (B)  -> batch_size pair
    sim_ji      = - self.cosine_sim(z[batch_size:], z[:batch_size])     # (B)  -> batch_size pair
    sim_pos     = tf.concat((sim_ij,sim_ji), axis=0)                    # (2B) -> 2*batch_size positive pair
    numerator   = tf.math.exp(sim_pos / self.temperature)               # (2B) -> 2*batch_size positive pair
    # print(numerator)
    sim_neg     = - self.cosine_sim(tf.expand_dims(z, 1), z)            # sim (Bx1xE, BxE) -> (2Bx2B)
    mask        = 1 - tf.eye(2*batch_size, dtype=tf.float32)            # (2Bx2B)
    sim_neg     = mask * tf.math.exp(sim_neg / self.temperature)        # (2Bx2B)
    denominator = tf.math.reduce_sum(sim_neg, axis=-1)                  # (2B) 
    # print(denominator)
    mean_cosine_similarity = tf.reduce_mean(- tf.math.log(numerator / denominator))       # () -> scalar
    tf.debugging.check_numerics(mean_cosine_similarity.numpy(), 'Cosine contains NaN values.')
    # print('cos: ', mean_cosine_similarity)

    # Compute total loss with associated rate
    total_loss = (1-self.rate)*mean_distance + self.rate*mean_cosine_similarity 
    # print('total: ',total_loss)
    tf.debugging.check_numerics(total_loss.numpy(), 'Total contains NaN values.')
    return total_loss

pre_trained_model =  tf.keras.models.load_model('/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_567_con_2loss/contrastive_model.h5',custom_objects={'ContrastiveLoss': ContrastiveLoss})
# pre_trained_model = tf.keras.models.load_model('/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_contrastive_2/contrastive_model.h5')

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv2D, Activation, Dropout, Flatten, Input, Dense, MultiHeadAttention
from tensorflow.keras.layers import Add, Average, Concatenate,Reshape, multiply, Permute, Lambda
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.activations import sigmoid

class Contrastive_Loss_2(tf.keras.losses.Loss):
  def __init__(self, temperature=0.5, rate=0.5, name='Contrastive_Loss_2', **kwargs):
    super(Contrastive_Loss_2, self).__init__(name=name, **kwargs)
    self.temperature   = temperature
    self.rate          = rate
    self.cosine_sim    = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.NONE)
    
  # @tf.function
  def call(self, z1, z2):
    batch_size, n_dim = z1.shape

    # Compute Euclid Distance loss
    difference    = z1 - z2                                             # (BxB)   * z1 and z2 already applied soft max -> in the last axis, max dif will be 1 
    squared_norm  = tf.reduce_sum(tf.square(difference), axis=1)        # (B)
    distance      = tf.sqrt(squared_norm + 1e-8)                        # (B)     * + epsilon to avoid Nan in gradient
    mean_distance = tf.reduce_mean(distance)                            # () -> scalar
    tf.debugging.check_numerics(mean_distance.numpy(), 'Distance contains NaN values.')
    # print('distance: , ',mean_distance)

    # Compute Consine Similarity loss
    z = tf.concat((z1, z2), 0)

    sim_ij      = - self.cosine_sim(z[:batch_size], z[batch_size:])     # (B)  -> batch_size pair
    sim_ji      = - self.cosine_sim(z[batch_size:], z[:batch_size])     # (B)  -> batch_size pair
    sim_pos     = tf.concat((sim_ij,sim_ji), axis=0)                    # (2B) -> 2*batch_size positive pair
    numerator   = tf.math.exp(sim_pos / self.temperature)               # (2B) -> 2*batch_size positive pair
  
    sim_neg     = - self.cosine_sim(tf.expand_dims(z, 1), z)            # sim (Bx1xE, BxE) -> (2Bx2B)
    mask        = 1 - tf.eye(2*batch_size, dtype=tf.float32)            # (2Bx2B)
    sim_neg     = mask * tf.math.exp(sim_neg / self.temperature)        # (2Bx2B)
    denominator = tf.math.reduce_sum(sim_neg, axis=-1)                  # (2B) 
  
    mean_cosine_similarity = tf.reduce_mean(- tf.math.log((numerator + 1e-11) / (denominator + 1e-11)))       # () -> scalar
    tf.debugging.check_numerics(mean_cosine_similarity.numpy(), 'Cosine contains NaN values.')
    # print('similarity: , ',mean_cosine_similarity)

    # Compute total loss with associated rate
    total_loss = (1-self.rate)*mean_distance + self.rate*mean_cosine_similarity 
    tf.debugging.check_numerics(total_loss.numpy(), 'Total contains NaN values.')
    return total_loss

def trippleAttention(x): # 8x8xc
  ## TA - Tripple Attention
  c = x.shape[-1]
    # channel
  tl1 = tf.math.reduce_mean(x, axis=-1) + tf.math.reduce_max(x, axis=-1)# 8x8        
  tl1 = MultiHeadAttention(num_heads=16, key_dim=8)(tl1, tl1)           # 8x8 
  tl1 = sigmoid(tl1)            # 8x8
  tl1 = Reshape((8,8,1))(tl1)   # 8x8x1
  tl1 = x * tl1                 # 8x8xc * 8x8x1 -> 8x8xc
    # width
  tl2 = tf.math.reduce_mean(x, axis=-2) + tf.math.reduce_max(x, axis=-2)# 8xc
  tl2 = MultiHeadAttention(num_heads=16, key_dim=8)(tl2, tl2)          # 8xc
  tl2 = sigmoid(tl2)            # 8xcx1
  tl2 = Reshape((8,1,c))(tl2)   # 8x1xc
  tl2 = x * tl2                 # 8x8xc * 8x1xc -> 8x8xc
    # height
  tl3 = tf.math.reduce_mean(x, axis=-3) + tf.math.reduce_max(x, axis=-3)# 8xc
  tl3 = MultiHeadAttention(num_heads=16, key_dim=8)(tl3, tl3)          # 8xc
  tl3 = sigmoid(tl3)            # 8xcx1
  tl3 = Reshape((1,8,c))(tl3)   # 8x1xc
  tl3 = x * tl3                 # 8x8xc * 1x8xc -> 8x8xc
    # average 
  t = Average()([tl1, tl2, tl3]) # 8x8xc
  t = GlobalAveragePooling2D(keepdims=False)(t) #channel
  return t # c

def get_model_architecture():
  base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(256,256,3))

  block7_x = base_model.output                           
  block6_x = base_model.get_layer('block6d_add').output  
  block5_x = base_model.get_layer('block5c_add').output  
  
  block6_x = Conv2D(filters=1280, kernel_size=1, strides=1)(block6_x) 
  block5_x = Conv2D(filters=1280, kernel_size=2, strides=2)(block5_x) 
  
  block7_x = trippleAttention(block7_x)
  block6_x = trippleAttention(block6_x)
  block5_x = trippleAttention(block5_x)
  
  x = block5_x + block6_x + block7_x
  x = Dense(512, 
                    activation='relu',
                    kernel_initializer=initializers.TruncatedNormal(mean=0.0,stddev=0.1),
                    kernel_regularizer=regularizers.l2(1e-5),
                    bias_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.1),
                    bias_regularizer=regularizers.l2(1e-5)
                    )(x)
  x = Dropout(0.2)(x)
  predictions = Dense(45, 
                        activation='softmax',
                        kernel_initializer=initializers.TruncatedNormal(mean=0.0,stddev=0.1),
                        kernel_regularizer=regularizers.l2(1e-5),
                        bias_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.1),
                        bias_regularizer=regularizers.l2(1e-5)
                        )(x)
  return Model(base_model.input, predictions)

# create full model
def get_cls_model(pre_trained_model):
  model = get_model_architecture()
  model.set_weights(pre_trained_model.get_weights()) 
  model.trainable = True 
  opt = tf.keras.optimizers.Adam(learning_rate=1e-7)  
  model.compile(loss=[tf.keras.losses.CategoricalCrossentropy(),Contrastive_Loss_2()], optimizer=opt, metrics=[])   
  return model

if os.path.exists('/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_567_con_2loss_2/contrastive_model.h5'):
  print('loading model !')
  model = tf.keras.models.load_model( filepath='/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_567_con_2loss_2/contrastive_model.h5', 
                                      custom_objects={'Contrastive_Loss_2': Contrastive_Loss_2})
  print(model.loss)
  print(model.optimizer.learning_rate)
  # model.summary()
else:
  print('creating model !')
  model = get_cls_model(pre_trained_model)
  # model.summary()


# Hyper-parameter.py

In [ ]:
class hypara(object):
  def __init__(self):
    self.label_dict  = dict(    airplane              = 0,
                                airport               = 1,
                                baseball_diamond      = 2,
                                basketball_court      = 3,
                                beach                 = 4,
                                bridge                = 5,
                                chaparral             = 6,
                                church                = 7,
                                circular_farmland     = 8,
                                cloud                 = 9,
                                commercial_area       = 10,
                                dense_residential     = 11,
                                desert                = 12,
                                forest                = 13,
                                freeway               = 14,
                                golf_course           = 15,
                                ground_track_field    = 16,
                                harbor                = 17,
                                industrial_area       = 18,
                                intersection          = 19,
                                island                = 20,
                                lake                  = 21,
                                meadow                = 22,
                                medium_residential    = 23,
                                mobile_home_park      = 24,
                                mountain              = 25,
                                overpass              = 26,
                                palace                = 27,
                                parking_lot           = 28,
                                railway               = 29,
                                railway_station       = 30,
                                rectangular_farmland  = 31,
                                river                 = 32,
                                roundabout            = 33,
                                runway                = 34,
                                sea_ice               = 35,
                                ship                  = 36,
                                snowberg              = 37,
                                sparse_residential    = 38,
                                stadium               = 39,
                                storage_tank          = 40,
                                tennis_court          = 41,
                                terrace               = 42,
                                thermal_power_station = 43,
                                wetland               = 44 )

        #---- Para for generator and training
    self.W             = 256   
    self.H             = 256  
    self.C             = 3    
    self.batch_size    = 45 
    self.learning_rate = 1e-6
    self.check_every   = 200
    self.class_num     = 45
    self.epoch_num     = 273

# Generator.py

## TrainGenerator

In [ ]:
import random
import numpy as np
import tensorflow as tf
from PIL import Image, ImageFilter, ImageEnhance
import os

class TrainGenerator(object):
  def __init__(self, img_dir, batch_size=15):  
    self.img_dir       = img_dir # 'content/dataset2/train/'
    self.batch_size    = batch_size
    self.class_num     = hypara().class_num
    self.label_dict    = hypara().label_dict
    
  def get_num_classes(self):
    return len(os.listdir(self.img_dir))

  def get_imgs_of_class(self,cls_name):
    return os.listdir(os.path.join(self.img_dir,cls_name))

  def get_batch (self, idx_num, class_list, is_aug=True):
    if self.batch_size != len(class_list):
      raise ValueError('batch_size is not equal !')
    
    x = np.zeros((self.batch_size*2,256,256,3), dtype=np.float32)
    y = np.zeros((self.batch_size*2,self.class_num), dtype=np.float32)
         
    i = 0
    for c in class_list: # class_list can be considered as batch_size
      file_path     = os.path.join(self.img_dir, c)
      file_list     = self.get_imgs_of_class(c)

      file_name_1   = file_list[idx_num]
      file_name_2   = file_list[random.randrange(140)]

      file_open_1   = os.path.join(file_path, file_name_1)
      file_open_2   = os.path.join(file_path, file_name_2)
     
      img_1         = Image.open(file_open_1) 
      img_2         = Image.open(file_open_2) 

      if is_aug == True:
        img_1       = self.augmentate(img_1)
        img_2       = self.augmentate(img_2) 

      img_1 = np.asarray(img_1).astype(np.float32) # 256x256x3
      img_2 = np.asarray(img_2).astype(np.float32) # 256x256x3

      # img_1 /= 255
      # img_2 /= 255
          
      x[i,:,:,:] = img_1
      x[i+self.batch_size,:,:,:] = img_2

      class_idx  = self.label_dict[c]
      y[i, class_idx] = 1
      y[i+self.batch_size, class_idx] = 1

      i += 1

    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)

    return x, y

  def augmentate(self, one_image): #---- data augmentation
    if np.random.uniform() > 0.6: # not augmentation
      return one_image

    if np.random.uniform() > 0.5: # rotate
      angle = random.choice([15,90,180,270])
      one_image  = one_image.rotate(angle)
    
    if np.random.uniform() > 0.5: # crop
      rand = np.random.randint(0,15)
      one_image  = one_image.crop((rand,rand,256-rand,256-rand))
      one_image  = one_image.resize((256,256), resample=Image.BILINEAR)
    
    rand = np.random.uniform()
    if rand > 0.5: # change color channel
      r, g, b = one_image.split()
      tup =  (b, r, g) if rand > 0.75 else (g, b, r)
      one_image = Image.merge("RGB", tup)
    
    if np.random.uniform() > 0.5: # change brightness
      one_image = one_image.point(lambda i: i * 1.17 if i*1.17 <= 255 else 255)
    
    if np.random.uniform() > 0.5: # add noise
      one_image = one_image.point(lambda i: i + np.random.randint(-4,4)) 

    rand = np.random.uniform() 
    if rand > 0.75: # change contrast/sharpness
      enhancer = ImageEnhance.Sharpness(one_image)
      one_image = enhancer.enhance(2.73)
    elif rand > 0.5:
      enhancer = ImageEnhance.Contrast(one_image)
      one_image = enhancer.enhance(2.27)

    return one_image

img_dir = '/content/dataset2/train/'
generator = TrainGenerator(img_dir=img_dir, batch_size=5)

print(generator.get_num_classes())
print(len(generator.get_imgs_of_class('roundabout')))

cls = ['chaparral', 'runway', 'sea_ice', 'intersection', 'desert']
x, y = generator.get_batch(0,cls,True)
print(x.shape,y.shape)
print(tf.argmax(y,axis=-1))

In [ ]:
img = Image.fromarray((x[0].numpy()).astype(np.uint8), 'RGB') # *255
img.show()
img = Image.fromarray((x[0+5].numpy()).astype(np.uint8), 'RGB') 
img.show()
img = Image.fromarray((x[1].numpy()).astype(np.uint8), 'RGB')
img.show()
img = Image.fromarray((x[1+5].numpy()).astype(np.uint8), 'RGB')
img.show()
img = Image.fromarray((x[2].numpy()).astype(np.uint8), 'RGB')
img.show()
img = Image.fromarray((x[2+5].numpy()).astype(np.uint8), 'RGB')
img.show()

del img, x, y, generator, img_dir, cls

## TestGenerator

In [ ]:
import random
import numpy as np
import tensorflow as tf
from PIL import Image, ImageFilter, ImageEnhance
import os

class TestGenerator(object):
  def __init__(self, img_dir):  
    self.img_dir       = img_dir # 'content/dataset2/test/'
    self.batch_size    = 45
    self.class_num     = hypara().class_num
    self.label_dict    = hypara().label_dict
    self.class_list    = list(self.label_dict.keys())

  def get_class_list(self):
    return self.class_list

  def get_num_classes(self):
    return self.class_num, len(self.class_list)

  def get_imgs_of_class(self, cls):
    file_list = os.listdir(os.path.join(self.img_dir, cls))
    file_list.sort()
    return file_list

  def get_batch (self, idx): 
    if self.class_num != len(self.class_list):
      raise ValueError('batch_size != 45')

    x = np.zeros((self.batch_size,256,256,3), dtype=np.float32)
    y = np.zeros((self.batch_size,self.class_num), dtype=np.float32)

    i = 0
    for c in self.class_list:
      file_path   = os.path.join(self.img_dir, c) 
      file_list   = self.get_imgs_of_class(c)
      file_name   = file_list[idx]
      file_open   = os.path.join(file_path, file_name)
      one_image   = Image.open(file_open) 
      one_image   = np.asarray(one_image).astype(np.float32) # 256x256x3
      # one_image  /= 255
      class_idx   = self.label_dict[c]

      x[i,:,:,:]  = one_image
      y[i, class_idx] = 1
      i += 1

    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)

    return x, y, i

test_dir = '/content/dataset2/test/'
test_generator = TestGenerator(img_dir=test_dir)

print(test_generator.get_num_classes())
print(test_generator.get_class_list())
print(len(test_generator.get_imgs_of_class('roundabout')),len(test_generator.get_imgs_of_class('bridge')))

x, y, n_imgs = test_generator.get_batch(idx=1)

print(x.shape,y.shape, n_imgs)
print(tf.argmax(y,axis=-1))

In [ ]:
img = Image.fromarray((x[0].numpy()).astype(np.uint8), 'RGB') # *255
img.show()
img = Image.fromarray((x[1].numpy()).astype(np.uint8), 'RGB') 
img.show()
img = Image.fromarray((x[2].numpy()).astype(np.uint8), 'RGB')
img.show()
img = Image.fromarray((x[3].numpy()).astype(np.uint8), 'RGB')
img.show()
img = Image.fromarray((x[4].numpy()).astype(np.uint8), 'RGB')
img.show()
img = Image.fromarray((x[5].numpy()).astype(np.uint8), 'RGB')
img.show()

del img, x, y, test_generator, test_dir

# Train.py

## Train.py for model_1

In [ ]:
import random
from datetime import datetime

def Accuracy(y_true, y_pred):
  if y_true.shape != y_pred.shape:
    print('Error metric !')
    raise ValueError('Something error in Acc calculation')
  y_true   = tf.argmax(y_true, axis=-1) # (B) 
  y_pred   = tf.argmax(y_pred, axis=-1) # (B)
  correct  = tf.cast(y_true == y_pred, tf.float32)
  accuracy = tf.reduce_sum(correct)
  return accuracy.numpy()

def lr_schedule(epoch, lr):
  if epoch < 25:
    return 1e-6
  elif epoch < 35:
    return 7.2e-7 
  elif epoch < 45:
    return 5e-7
  elif epoch < 55:
    return 2e-6
  elif epoch < 96:
    return 1e-6
  else:
    return 9e-7

TRAIN_DIR       = '/content/dataset2/train/'
TEST_DIR        = '/content/dataset2/test/'
stored_dir      = '/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_567_con_2loss'
best_model_file = '/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_567_con_2loss/contrastive_model.h5'
BATCH_SIZE      = 15 # 3 or 5 or 9 or 15 or 45
alpha           = 0.72 # tuning parameter
current_epoch   = 116
old_test_acc    = 0.9381746031746032
train_generator = TrainGenerator(img_dir=TRAIN_DIR, batch_size=BATCH_SIZE)
test_generator  = TestGenerator(img_dir=TEST_DIR)


for epoch in range(current_epoch, 127):
  print('\n\n ==================== Epoch: ', epoch,'======================')
  s = datetime.now()
  train_acc = 0
  epoch_loss = 0
  label_dict = hypara().label_dict
  class_list = list(label_dict.keys())
  random.shuffle(class_list)

  # if epoch == current_epoch:
  #   model.optimizer.learning_rate = 1e-6
  # else:  
  model.optimizer.learning_rate = lr_schedule(epoch, model.optimizer.learning_rate.numpy())
  print(' *** learning rate: ', model.optimizer.learning_rate)
  print('-------- training ---------')
  for i in range(int(45 / BATCH_SIZE)):
    for n_batch_train in range(140):
      x_train, y_true_train = train_generator.get_batch(idx_num=n_batch_train, class_list=class_list[i*BATCH_SIZE:(i+1)*BATCH_SIZE], is_aug=True) # return 2 batches of images, each batch contain B images from B class
      with tf.GradientTape() as tape:
        y_pred_train = model(x_train)
        train_acc   += Accuracy(y_true_train, y_pred_train)
        # print(Accuracy(y_true_train, y_pred_train)/30)
        loss_1       = model.loss[0](y_true_train, y_pred_train)                            # categorical cross entropy
        loss_2       = model.loss[1](y_pred_train[:BATCH_SIZE], y_pred_train[BATCH_SIZE:])  # probability distance + consine similairy
        loss         = alpha*loss_1 + (1-alpha)*loss_2                                      # total loss 
        epoch_loss  += loss 
        grads        = tape.gradient(loss, model.trainable_variables) 
        tf.debugging.check_numerics(grads[0], 'grad contains NaN values.')
        model.optimizer.apply_gradients(zip(grads, model.trainable_variables))  

  train_acc /= 6300*2
  print('# epoch loss:  ', epoch_loss.numpy(), '; epoch acc: ', train_acc)
  
  print("------ testing -------")
  if epoch >= 0: # 5
    start_test   = datetime.now()
    test_acc     = 0
    img_test     = 0
    for n_batch_test in range(560):
      x_test, y_true_test, n_imgs = test_generator.get_batch(n_batch_test)
      y_pred_test         = model(x_test)
      test_acc           += Accuracy(y_true_test, y_pred_test)
      img_test += n_imgs
    print(img_test)
    test_acc /= 25200
    print('# test accuray: ', test_acc, '  and time needed for test: ', datetime.now()-start_test)
       # Save model when successfully testing
    if (test_acc > old_test_acc): 
      old_test_acc = test_acc
      model.save(best_model_file)
      print('Save model completed')
      with open(os.path.join(stored_dir,"train_log.txt"), "a") as text_file:
        text_file.write("Save best model at Epoch: {}; Accuracy: {}\n".format(epoch, old_test_acc))

  with open(os.path.join(stored_dir,"train_log.txt"), "a") as text_file:
    text_file.write("Epoch: {}; lr: {}; Train accuracy: {}\n".format(epoch, model.optimizer.learning_rate.numpy(), train_acc))
  print('# epoch training time: ', datetime.now()-s, '\n')


In [ ]:
print(loss_1.numpy(), loss_2.numpy(), epoch_loss.numpy(), test_acc )

## Train.py for model_2

In [ ]:
import random
from datetime import datetime

def Accuracy(y_true, y_pred):
  if y_true.shape != y_pred.shape:
    print('Error metric !')
    raise ValueError('Something error in Acc calculation')
  y_true   = tf.argmax(y_true, axis=-1) # (B) 
  y_pred   = tf.argmax(y_pred, axis=-1) # (B)
  correct  = tf.cast(y_true == y_pred, tf.float32)
  accuracy = tf.reduce_sum(correct)
  return accuracy.numpy()

def lr_schedule(epoch, lr):
  if epoch < 6:
    return 1e-7
  elif epoch < 15:
    return 6e-8 
  elif epoch < 20:
    return 2.7e-8
  else:
    return 1e-8

TRAIN_DIR       = '/content/dataset2/train/'
TEST_DIR        = '/content/dataset2/test/'
stored_dir      = '/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_567_con_2loss_2'
best_model_file = '/content/drive/MyDrive/RSIC/NWPU-RESISC45/effb0_567_con_2loss_2/contrastive_model.h5'
BATCH_SIZE      = 15  # 3 or 5 or 9 or 15 or 45
alpha           = 0.6 # tuning parameter
current_epoch   = 32
old_test_acc    = 0.9383333333333334
train_generator = TrainGenerator(img_dir=TRAIN_DIR, batch_size=BATCH_SIZE)
test_generator  = TestGenerator(img_dir=TEST_DIR)


for epoch in range(current_epoch, 127):
  print('\n\n ==================== Epoch: ', epoch,'======================')
  s = datetime.now()
  train_acc = 0
  epoch_loss = 0
  label_dict = hypara().label_dict
  class_list = list(label_dict.keys())
  random.shuffle(class_list)

  # if epoch == current_epoch:
  #   model.optimizer.learning_rate = 1e-6
  # else:  
  model.optimizer.learning_rate = lr_schedule(epoch, model.optimizer.learning_rate.numpy())
  print(' *** learning rate: ', model.optimizer.learning_rate)
  print('-------- training ---------')
  for i in range(int(45 / BATCH_SIZE)):
    for n_batch_train in range(140):
      x_train, y_true_train = train_generator.get_batch(idx_num=n_batch_train, class_list=class_list[i*BATCH_SIZE:(i+1)*BATCH_SIZE], is_aug=True) # return 2 batches of images, each batch contain B images from B class
      with tf.GradientTape() as tape:
        y_pred_train = model(x_train)
        train_acc   += Accuracy(y_true_train, y_pred_train)
        loss_1       = model.loss[0](y_true_train, y_pred_train)                            # categorical cross entropy
        loss_2       = model.loss[1](y_pred_train[:BATCH_SIZE], y_pred_train[BATCH_SIZE:])  # euclid distance + consine similairy
        loss         = alpha*loss_1 + (1-alpha)*loss_2                                      # total loss 
        epoch_loss  += loss 
        grads        = tape.gradient(loss, model.trainable_variables) 
        tf.debugging.check_numerics(grads[0], 'grad contains NaN values.')
        model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_acc /= 6300*2
  print('# epoch loss:  ', epoch_loss.numpy(), '; epoch acc: ', train_acc)
  
  print("------ testing -------")
  if epoch >= 0: 
    start_test   = datetime.now()
    test_acc     = 0
    img_test     = 0
    for n_batch_test in range(560):
      x_test, y_true_test, n_imgs = test_generator.get_batch(n_batch_test)
      y_pred_test         = model(x_test)
      test_acc           += Accuracy(y_true_test, y_pred_test)
      img_test += n_imgs
    print(img_test)
    test_acc /= 25200
    print('# test accuray: ', test_acc, '  and time needed for test: ', datetime.now()-start_test)
       # Save model when successfully testing
    if (test_acc > old_test_acc): 
      old_test_acc = test_acc
      model.save(best_model_file)
      print('Save model completed')
      with open(os.path.join(stored_dir,"train_log.txt"), "a") as text_file:
        text_file.write("Save best model at Epoch: {}; Accuracy: {}\n".format(epoch, old_test_acc))

  with open(os.path.join(stored_dir,"train_log.txt"), "a") as text_file:
    text_file.write("Epoch: {}; lr: {}; Train accuracy: {}\n".format(epoch, model.optimizer.learning_rate.numpy(), train_acc))
  print('# epoch training time: ', datetime.now()-s, '\n')


In [ ]:
print(loss_1.numpy(), loss_2.numpy(),loss.numpy(), epoch_loss.numpy())